In [1]:
import dask.dataframe as dd
import dask.bag as da
import os
import json
import dask

from dask.distributed import LocalCluster
cluster = LocalCluster()          # Fully-featured local Dask cluster
client = cluster.get_client()

In [2]:
cluster.dashboard_link

'http://127.0.0.1:8787/status'

In [3]:
import pandas as pd

In [4]:
pokemon_df = pd.read_csv('Files/pokemonDB_dataset.csv')
pokemon_df.head()

,Pokemon,Type,Species,Height,Weight,Abilities,EV Yield,Catch Rate,Base Friendship,Base Exp,...,Defense Max,Special Attack Base,Special Attack Min,Special Attack Max,Special Defense Base,Special Defense Min,Special Defense Max,Speed Base,Speed Min,Speed Max
0,Abomasnow,"Grass, Ice",Frost Tree Pokémon,2.2 m (7′03″),135.5 kg (298.7 lbs),"1. Snow Warning, Soundproof (hidden ability)","1 Attack, 1 Sp. Atk","60 (7.8% with PokéBall, full HP)",50 (normal),173,...,273,92,170,311,85,157,295,60,112,240
1,Mega Abomasnow,"Grass, Ice",Frost Tree Pokémon,2.7 m (8′10″),185.0 kg (407.9 lbs),1. Snow Warning,"1 Attack, 1 Sp. Atk","60 (7.8% with PokéBall, full HP)",50 (normal),208,...,339,132,242,399,105,193,339,30,58,174
2,Abra,Psychic,Psi Pokémon,0.9 m (2′11″),19.5 kg (43.0 lbs),"1. Synchronize, 2. Inner Focus, Magic Guard (h...",1 Sp. Atk,"200 (26.1% with PokéBall, full HP)",50 (normal),62,...,141,105,193,339,55,103,229,90,166,306
3,Absol,Dark,Disaster Pokémon,1.2 m (3′11″),47.0 kg (103.6 lbs),"1. Pressure, 2. Super Luck, Justified (hidden ...",2 Attack,"30 (3.9% with PokéBall, full HP)",35 (lower than normal),163,...,240,75,139,273,60,112,240,75,139,273
4,Mega Absol,Dark,Disaster Pokémon,1.2 m (3′11″),49.0 kg (108.0 lbs),1. Magic Bounce,2 Attack,"30 (3.9% with PokéBall, full HP)",35 (lower than normal),198,...,240,115,211,361,60,112,240,115,211,361


In [5]:
trainers = dd.read_parquet('Files/trainers_with_pockemon.parquet')
trainers

,first_name,last_name,age,pockemon
npartitions=16,,,,
,string,string,int64,string
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


In [6]:
trainers.head()

,first_name,last_name,age,pockemon
0,Casey,Sheppard,48,Combee
1,Arturo,Blake,108,Chewtle
2,Tamiko,Riddle,113,Wailmer
3,Tamiko,Riddle,113,Impidimp
4,Tamiko,Riddle,113,Natu


In [7]:
trainers_with_pokemon = trainers.merge(pokemon_df, right_on="Pokemon", left_on="pockemon", how="inner")
trainers_with_pokemon

,first_name,last_name,age,pockemon,Pokemon,Type,Species,Height,Weight,Abilities,EV Yield,Catch Rate,Base Friendship,Base Exp,Growth Rate,Egg Groups,Gender,Egg Cycles,HP Base,HP Min,HP Max,Attack Base,Attack Min,Attack Max,Defense Base,Defense Min,Defense Max,Special Attack Base,Special Attack Min,Special Attack Max,Special Defense Base,Special Defense Min,Special Defense Max,Speed Base,Speed Min,Speed Max
npartitions=16,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,string,string,int64,string,string,string,string,string,string,string,string,string,string,string,string,string,string,string,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


3. **Trainer with highest mean defense**

In [8]:
mean_defense = trainers_with_pokemon.groupby(["first_name", "last_name"]).agg({"Defense Max": "mean"}).sort_values(["Defense Max"], ascending=False)
mean_defense

,Defense Max
npartitions=1,
,float64
,...


In [9]:
mean_defense.compute()

,,Defense Max
first_name,last_name,
Nicolas,Ballard,658.0
Christian,Roy,658.0
Jasper,Albert,658.0
Phylicia,Maddox,658.0
Shanice,Ortega,658.0
...,...,...
Kory,Morse,119.0
Bradly,Gates,119.0
Gale,Reese,119.0


4. **Youngest trainer with a fire pockemon**

In [10]:
trainers_with_pokemon.head()

,first_name,last_name,age,pockemon,Pokemon,Type,Species,Height,Weight,Abilities,...,Defense Max,Special Attack Base,Special Attack Min,Special Attack Max,Special Defense Base,Special Defense Min,Special Defense Max,Speed Base,Speed Min,Speed Max
0,Casey,Sheppard,48,Combee,Combee,"Bug, Flying",Tiny Bee Pokémon,0.3 m (1′00″),5.5 kg (12.1 lbs),"1. Honey Gather, Hustle (hidden ability)",...,201,30,58,174,42,80,201,70,130,262
1,Arturo,Blake,108,Chewtle,Chewtle,Water,Snapping Pokémon,0.3 m (1′00″),8.5 kg (18.7 lbs),"1. Strong Jaw, 2. Shell Armor, Swift Swim (hid...",...,218,38,72,192,38,72,192,44,83,205
2,Tamiko,Riddle,113,Wailmer,Wailmer,Water,Ball Whale Pokémon,2.0 m (6′07″),130.0 kg (286.6 lbs),"1. Water Veil, 2. Oblivious, Pressure (hidden ...",...,185,70,130,262,35,67,185,60,112,240
3,Tamiko,Riddle,113,Impidimp,Impidimp,"Dark, Fairy",Wily Pokémon,0.4 m (1′04″),5.5 kg (12.1 lbs),"1. Prankster, 2. Frisk, Pickpocket (hidden abi...",...,174,55,103,229,40,76,196,50,94,218
4,Tamiko,Riddle,113,Natu,Natu,"Psychic, Flying",Tiny Bird Pokémon,0.2 m (0′08″),2.0 kg (4.4 lbs),"1. Synchronize, 2. Early Bird, Magic Bounce (h...",...,207,70,130,262,45,85,207,70,130,262


In [11]:
triners_with_fire_pockemon = trainers_with_pokemon.loc[trainers_with_pokemon["Type"] == "Fire"]
min_age = triners_with_fire_pockemon["age"].min().compute()
min_age

0

In [12]:
triners_with_fire_pockemon.loc[trainers_with_pokemon.age == min_age].compute()

,first_name,last_name,age,pockemon,Pokemon,Type,Species,Height,Weight,Abilities,...,Defense Max,Special Attack Base,Special Attack Min,Special Attack Max,Special Defense Base,Special Defense Min,Special Defense Max,Speed Base,Speed Min,Speed Max
2373,Derek,Byers,0,Simisear,Simisear,Fire,Ember Pokémon,1.0 m (3′03″),28.0 kg (61.7 lbs),"1. Gluttony, Blaze (hidden ability)",...,247,98,180,324,63,117,247,101,186,331
30693,Lyle,Stone,0,Darmanitan Standard Mode,Darmanitan Standard Mode,Fire,Blazing Pokémon,1.3 m (4′03″),92.9 kg (204.8 lbs),"1. Sheer Force, Zen Mode (hidden ability)",...,229,30,58,174,55,103,229,95,175,317
32455,Piedad,Barker,0,Fuecoco,Fuecoco,Fire,Fire Croc Pokémon,0.4 m (1′04″),9.8 kg (21.6 lbs),"1. Blaze, Unaware (hidden ability)",...,238,63,117,247,40,76,196,36,69,188
36070,Winter,Kelly,0,Darumaka,Darumaka,Fire,Zen Charm Pokémon,0.6 m (2′00″),37.5 kg (82.7 lbs),"1. Hustle, Inner Focus (hidden ability)",...,207,15,31,141,45,85,207,50,94,218
39687,Odell,Bates,0,Chimchar,Chimchar,Fire,Chimp Pokémon,0.5 m (1′08″),6.2 kg (13.7 lbs),"1. Blaze, Iron Fist (hidden ability)",...,205,58,108,236,44,83,205,61,114,243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383927,Lou,Morrison,0,Magby,Magby,Fire,Live Coal Pokémon,0.7 m (2′04″),21.4 kg (47.2 lbs),"1. Flame Body, Vital Spirit (hidden ability)",...,190,70,130,262,55,103,229,83,153,291
387801,Oda,Nolan,0,Litten,Litten,Fire,Fire Cat Pokémon,0.4 m (1′04″),4.3 kg (9.5 lbs),"1. Blaze, Intimidate (hidden ability)",...,196,60,112,240,40,76,196,70,130,262
390570,Shalanda,Avery,0,Crocalor,Crocalor,Fire,Fire Croc Pokémon,1.0 m (3′03″),30.7 kg (67.7 lbs),"1. Blaze, Unaware (hidden ability)",...,280,90,166,306,58,108,236,49,92,216
402902,Sal,Anderson,0,Castform Sunny Form,Castform Sunny Form,Fire,Weather Pokémon,0.3 m (1′00″),0.8 kg (1.8 lbs),1. Forecast,...,262,70,130,262,70,130,262,70,130,262


5. **Who is the trainer that has the higher number of water type Pockemon in his team?**

In [14]:
trainers_with_water_pockemon = trainers_with_pokemon.loc[trainers_with_pokemon["Type"] == "Water"]

In [17]:
pockemon_count = trainers_with_water_pockemon.groupby(["first_name", "last_name"]).agg({"Pokemon": "count"}).sort_values(["Pokemon"], ascending=False)
pockemon_count.head()

,,Pokemon
first_name,last_name,
Bobbie,Hurley,12
Trinity,Powell,10
Audrea,Guthrie,10
Glynda,Stephenson,10
Penni,Forbes,10


6. **How many coaches have a Psyduck on their team?**

In [23]:
trainers_with_psyduck = trainers_with_pokemon.loc[trainers_with_pokemon["Pokemon"] == "Psyduck"]
trainers_with_psyduck.compute().shape[0]

4825

7. **By the rule of the Pockemon league a trainer can carry a team of maximum 6 Pokemon, How many trainer are carrying an illegal number of pokemon?**

In [27]:
pockemon_count.loc[pockemon_count["Pokemon"] > 6].compute().shape[0]

32